In [67]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import math
import ast
import json

In [2]:
data = pd.read_csv('hh_metric_df.csv', sep=';') 
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


In [3]:
# будем парсить опыт работы

In [4]:
data.iloc[0]['Опыт работы']

'Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники'

In [5]:
data.iloc[1]['Опыт работы']

'Опыт работы 19 лет 5 месяцев  Январь 2000 — по настоящее время 19 лет 5 месяцев Временный трудовой коллектив Москва Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Менеджер проекта, Аналитик, Технический писатель Весь цикл ПО, кроме программирования.'

In [6]:
data.iloc[2]['Опыт работы']

'Опыт работы 10 лет 3 месяца  Октябрь 2004 — Декабрь  2014 10 лет 3 месяца ПАО Сбербанк Кассир-операционист Работа с денежной наличностью, обслуживание клиентов, операции с иностранной валютой, проведение инкассации, составление и обработка первичных документов.'

In [8]:
data.isna().sum()

Пол, возраст                         0
ЗП                                   0
Ищет работу на должность:            0
Город, переезд, командировки         0
Занятость                            0
График                               0
Опыт работы                        168
Последнее/нынешнее место работы      1
Последняя/нынешняя должность         2
Образование и ВУЗ                    0
Обновление резюме                    0
Авто                                 0
dtype: int64

In [9]:
data.dropna(subset=['Опыт работы'], inplace=True)

In [10]:
data[data['Опыт работы'] == 'Не указано']

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
19199,"Мужчина , 39 лет , родился 17 апреля 1980",45000 руб.,системный администратор / инженер программист,"Пермь , готов к переезду , готов к редким кома...",полная занятость,полный день,Не указано,"АО ""Предприятие В-1336""",Инженер-программист,Высшее образование (Бакалавр) 2016 Пермский н...,10.04.2019 07:12,Не указано
26781,"Мужчина , 25 лет , родился 21 мая 1993",30000 руб.,Системный администратор,"Белгород , готов к переезду (Москва, Санкт-Пет...",полная занятость,"гибкий график, полный день, сменный график, ва...",Не указано,"Европа, Торговая сеть",Системный администратор,Высшее образование 2015 Белгородский государс...,10.04.2019 07:11,Имеется собственный автомобиль


In [11]:
data = data[data['Опыт работы'] != 'Не указано']

In [12]:
def remove_duplicate_experience(text):
    
    pattern = r'(Опыт работы(?:\s+\S+){0,6}\s+(?:года|год|лет|месяца|месяцев|месяц))'
    
    experience_list = re.split(pattern, text)
    
    experience_list = [elem.strip() for elem in experience_list if elem.strip()]
    combined_list = [experience_list[i] + ' ' + experience_list[i+1] \
                     for i in range(0, len(experience_list) - 1, 2)]
    
    return combined_list

In [13]:
def process_full_experience(text):
    
    combined_list = remove_duplicate_experience(text)
    
    if len(combined_list) > 1:
        return combined_list[1]
    elif len(combined_list) == 1:
        return combined_list[0]
    else:
        return np.nan

In [14]:
data['experience_clean'] = data['Опыт работы'].apply(process_full_experience)

In [15]:
data.iloc[0]['experience_clean']

'Опыт работы 16 лет 10 месяцев Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники'

In [16]:
def clean_experience_list(experience_list):
    
    pattern = r'\b\d+\s+(?:года|год|лет|месяца|месяцев|месяц)\b'
    
    cleaned_experience_list = []
    
    for elem in experience_list:
        cleaned_elem = re.sub(pattern, '', elem).strip()
        cleaned_experience_list.append(cleaned_elem)
        
    return cleaned_experience_list

In [17]:
def get_experience_len(exp):

    common_exp = 0
    
    cleaned_exp = re.sub(r'Опыт работы', '', exp).strip().split()

    for i, elem in enumerate(cleaned_exp):
        if elem in ['год', 'года', 'лет']:
            common_exp += int(cleaned_exp[i - 1]) * 12
        elif elem in ['месяц', 'месяца', 'месяцев']:
            common_exp += int(cleaned_exp[i - 1])
        elif not elem.isdigit():
            return np.nan
    
    return common_exp

In [18]:
def get_workExperience(x):
    
    months = get_experience_len(x)
    
    if math.isnan(months):
        return np.nan
    
    years = np.ceil(months / 12)

    if years == 0:
        return 'noExperience'
    elif 1 <= years < 3:
        return 'between1And3'
    elif 3 <= years < 6:
        return 'between3And6'
    else:
        return 'moreThan6'

In [19]:
common_experience = 'Опыт работы 16 лет 10 месяцев'
get_workExperience(common_experience)

'moreThan6'

In [20]:
def process_experience(row):
    
    pattern = (
        r'\b(?:Январь|Февраль|Март|Апрель|Май|Июнь|Июль|Август|'
        r'Сентябрь|Октябрь|Ноябрь|Декабрь)\s+\d{4}\s*[—-]\s*'
        r'(?:\b(?:Январь|Февраль|Март|Апрель|Май|Июнь|Июль|Август|'
        r'Сентябрь|Октябрь|Ноябрь|Декабрь)\s+\d{4}\b|по\s+настоящее\s+время|настоящее\s+время)\b'
    )

    text = row['experience_clean']

    experience_list = re.split(f'({pattern})', text)
    
    if len(experience_list) == 1:
        return np.nan

    experience_list = [elem.strip() for elem in experience_list if elem.strip()]

    cleaned_experience_list = clean_experience_list(experience_list)

    work_experience = get_workExperience(experience_list[0])
    
    try:
        experience_res = [cleaned_experience_list[i + 1] for i in range(1, len(cleaned_experience_list), 2)]
    except IndexError:
        return np.nan
    
    return pd.Series({'workExperience': work_experience, 'experience': experience_res})

tqdm.pandas()

data[['workExperience', 'experience']] = data.progress_apply(process_experience, axis=1)

100%|███████████████████████████████████| 44574/44574 [00:11<00:00, 3744.45it/s]


In [21]:
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,experience_clean,workExperience,experience
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 16 лет 10 месяцев Август 2010 — по...,moreThan6,"[МАОУ ""СОШ № 1 г.Немана"" Системный администрат..."
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,Опыт работы 19 лет 5 месяцев Январь 2000 — по ...,moreThan6,[Временный трудовой коллектив Москва Информаци...
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Опыт работы 10 лет 3 месяца Октябрь 2004 — Дек...,moreThan6,[ПАО Сбербанк Кассир-операционист Работа с ден...
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,Опыт работы 18 лет 9 месяцев Август 2017 — Апр...,moreThan6,"[OpenSoft Саратов , www.opensoftdev.ru Информа..."
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано,Опыт работы 5 лет 7 месяцев Август 201,NaN,NaN


In [22]:
data.loc[0]['experience']

['МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.',
 'ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники']

In [23]:
data.dropna(subset=['experience', 'workExperience'], inplace=True)

In [24]:
data.shape

(42001, 15)

In [25]:
vacancies = pd.read_csv('vacancies_full_keyskills.csv')
vacancies.head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
0,v_862116,Смотритель музейный,c_162972,"['Уверенный пользователь пк', 'Работа в команд...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1,Уверенный пользователь пк Работа в команде Уме...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1,Активные продажи Холодные продажи Кредитные до...
2,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2,Продуктивность Клиентоориентированность Работа...
3,v_2323706,Менеджер по качеству,c_244141,"['Контроль качества', 'Производственный контро...",88000.0,NaN,RUR,a_5920,ar_64,full,fullDay,between1And3,Менеджер по качеству,66,5469,160161,5,5,2,Контроль качества Производственный контроль На...
4,v_704266,Менеджер по продажам / Sales manager,c_192038,"['Активные продажи', 'Поиск и привлечение клие...",30000.0,100000.0,RUR,a_5398,ar_71,full,fullDay,between1And3,Менеджер по продажам,74,4888,102268,5,5,2,Активные продажи Поиск и привлечение клиентов ...


In [26]:
vacancies['keySkills'] = vacancies['keySkills'].str.strip('[]')
vacancies['keySkills'] = vacancies['keySkills'].str.split(',')
vacancies['keySkills'] = vacancies['keySkills']\
    .apply(lambda x: [skill.strip().strip('\'').capitalize() for skill in x]\
           if isinstance(x, list) else np.nan)
vacancies['keySkills']

0          [Уверенный пользователь пк, Работа в команде, ...
1          [Активные продажи, Холодные продажи, Кредитные...
2          [Продуктивность, Клиентоориентированность, Раб...
3          [Контроль качества, Производственный контроль,...
4          [Активные продажи, Поиск и привлечение клиенто...
                                 ...                        
2209567    [Внимательность к мелочам, Сборка заказов, Упа...
2209568    [Проверка сроков годности, Работа в команде, М...
2209569    [Маркетинговый анализ, Анализ рынка, Мониторин...
2209570    [Монтаж оборудования, Монтаж оборудования, Сле...
2209571    [1с: предприятие 8, Банк-клиент, 1с: бухгалтер...
Name: keySkills, Length: 2209572, dtype: object

In [27]:
unique_skills = set()

for skills in vacancies['keySkills']:
    unique_skills.update(skills)

In [28]:
len(unique_skills)

4658

In [44]:
unique_skills.remove('')

In [29]:
unique_names = set()

for name in vacancies['name']:
    unique_names.add(name)

In [30]:
len(unique_names)

724240

In [49]:
def extract_vacancy_name(text, unique_names):

    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in unique_names) + r')\b'
    matches = re.findall(pattern, text)
    names = list(dict.fromkeys(matches))
    
    return names

In [50]:
extract_vacancy_name(data.iloc[0]['experience'][0], unique_names)

['1', 'Системный администратор']

In [51]:
def extract_info(text, unique_list):
    
    pattern = r'\b(?:' + '|'.join(re.escape(unique_elem) for unique_elem in unique_list) + r')\b'
    matches = re.findall(pattern, text)
    
    return list(dict.fromkeys(matches))

In [52]:
def extract_vac_data(text, unique_names, unique_skills, salary, work_experience, employment):

    names = extract_info(text, unique_names)
    skills = extract_info(text, unique_skills)
    salary = salary
    work_experience = work_experience
    employment = employment

    vac_dict = {
        'name': names,
        'keyskills': skills,
        'salary': salary,
        'workExperience': work_experience,
        'employment': employment
    }
    
    return vac_dict

In [58]:
data_part = data[:500].copy()

In [59]:
tqdm.pandas()

data_part['experience_dict'] = data_part.progress_apply(
    lambda row: [extract_data_from_text(text,
                                        unique_names,
                                        unique_skills,
                                        row['ЗП'],
                                        row['workExperience'], 
                                        row['График']) for text in row['experience']], axis=1
)

100%|███████████████████████████████████████| 500/500 [4:45:59<00:00, 34.32s/it]


In [68]:
data_part.iloc[4]['experience_dict']

[{'name': ['Слесарь'],
  'keyskills': ['Розничная сеть', 'Продукты питания', 'Слесарь'],
  'salary': '25000 руб.',
  'workExperience': 'moreThan6',
  'employment': 'полный день, сменный график'},
 {'name': ['Слесарь',
   'Пользователь ПК',
   'Работа',
   'Техник АСУ ТП',
   'Наладчик КИПиА'],
  'keyskills': ['Розничная сеть',
   'Продукты питания',
   'Слесарь',
   'Грамотная речь',
   'Работа с оргтехникой',
   'Воронеж',
   'Автоматизация технологических процессов',
   'Наладчик'],
  'salary': '25000 руб.',
  'workExperience': 'moreThan6',
  'employment': 'полный день, сменный график'}]

In [63]:
data_part.to_csv('data_part.csv', index=False)

In [45]:
data = pd.read_csv('data_part.csv')
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,experience_clean,workExperience,experience,experience_dict
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 16 лет 10 месяцев Август 2010 — по...,moreThan6,"['МАОУ ""СОШ № 1 г.Немана"" Системный администра...","[{'name': ['1', 'Системный администратор'], 'k..."
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,Опыт работы 19 лет 5 месяцев Январь 2000 — по ...,moreThan6,['Временный трудовой коллектив Москва Информац...,"[{'name': ['Менеджер', 'Аналитик', 'Технически..."
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Опыт работы 10 лет 3 месяца Октябрь 2004 — Дек...,moreThan6,['ПАО Сбербанк Кассир-операционист Работа с де...,"[{'name': ['Кассир-операционист', 'Работа'], '..."
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,Опыт работы 18 лет 9 месяцев Август 2017 — Апр...,moreThan6,"['OpenSoft Саратов , www.opensoftdev.ru Информ...","[{'name': ['Инженер-программист'], 'keyskills'..."
4,"Мужчина , 29 лет , родился 5 октября 1989",25000 руб.,Технический специалист,"Старый Оскол , не готов к переезду , не готов ...",полная занятость,"полный день, сменный график",Опыт работы 9 лет 9 месяцев Технический специ...,Комбинат хлебопродуктов Старооскольский,Слесарь КИПиА,Неоконченное высшее образование 2013 Воронежс...,22.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 9 лет 9 месяцев Август 2009 — по н...,moreThan6,['Комбинат хлебопродуктов Старооскольский www....,"[{'name': ['Слесарь'], 'keyskills': ['Рознична..."


In [46]:
def filter_empty_name(experience):
    
    global removed_cnt
    global remaining_cnt
    
    try:
        experience_list = ast.literal_eval(experience)
    except (ValueError, SyntaxError):
        return experience
    
    if not isinstance(experience_list, list):
        return experience
    
    filtered_list = [d for d in experience_list if isinstance(d, dict) \
                     and isinstance(d.get('name'), list) and len(d.get('name')) > 0]
    removed_cnt += (len(experience_list) - len(filtered_list))
    remaining_cnt += len(filtered_list)
    
    return filtered_list

In [47]:
removed_cnt = 0
remaining_cnt = 0

tqdm.pandas()
data['experience_dict'] = data['experience_dict'].progress_apply(filter_empty_name)

print(f'Dicts removed: {removed_cnt}')
print(f'Dicts remaining: {remaining_cnt}')

100%|███████████████████████████████████████| 500/500 [00:00<00:00, 5707.75it/s]

Dicts removed: 199
Dicts remaining: 1912


In [48]:
type(data.iloc[0]['experience_dict'])

list

In [49]:
type(data.iloc[0]['experience_dict'][0])

dict

In [55]:
def clean_single_letter(experience_list):
    
    for d in experience_list:
        if 'name' in d:
            d['name'] = [name for name in d['name'] if len(name) > 1]
        if 'keyskills' in d:
            d['keyskills'] = [skill for skill in d['keyskills'] if len(skill) > 1 or skill == 'R']
            
    return experience_list 

In [56]:
data['experience_dict'] = data['experience_dict'].apply(clean_single_letter)

In [58]:
data.iloc[5]['experience_dict']

[{'name': ['Менеджер', 'Руководитель ИТ'],
  'keyskills': ['Москва',
   'Информационные технологии',
   'Системная интеграция',
   'R'],
  'salary': '250000 руб.',
  'workExperience': 'moreThan6',
  'employment': 'полный день'},
 {'name': ['Заместитель директора'],
  'keyskills': [],
  'salary': '250000 руб.',
  'workExperience': 'moreThan6',
  'employment': 'полный день'},
 {'name': ['Руководитель ИТ'],
  'keyskills': ['Информационные технологии',
   'Системная интеграция',
   'Управление проектами',
   'Руководство коллективом',
   'Заключение договоров',
   'Ведение переговоров',
   'Проведение презентаций',
   'Высшее образование',
   'Москва',
   'Тренинги',
   'Делегирование'],
  'salary': '250000 руб.',
  'workExperience': 'moreThan6',
  'employment': 'полный день'}]

In [59]:
start_len = len(data)

data = data[data['experience_dict'].apply(lambda x: len(x) > 0)]
removed_len = start_len - len(data)

print(f'Removed rows cnt: {removed_len}')

Removed rows cnt: 7


In [61]:
data.to_csv('data_part_clean.csv', index=False)

In [80]:
data = pd.read_csv('data_part_clean.csv')
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,experience_clean,workExperience,experience,experience_dict
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 16 лет 10 месяцев Август 2010 — по...,moreThan6,"['МАОУ ""СОШ № 1 г.Немана"" Системный администра...","[{'name': ['Системный администратор'], 'keyski..."
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,Опыт работы 19 лет 5 месяцев Январь 2000 — по ...,moreThan6,['Временный трудовой коллектив Москва Информац...,"[{'name': ['Менеджер', 'Аналитик', 'Технически..."
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Опыт работы 10 лет 3 месяца Октябрь 2004 — Дек...,moreThan6,['ПАО Сбербанк Кассир-операционист Работа с де...,"[{'name': ['Кассир-операционист', 'Работа'], '..."
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,Опыт работы 18 лет 9 месяцев Август 2017 — Апр...,moreThan6,"['OpenSoft Саратов , www.opensoftdev.ru Информ...","[{'name': ['Инженер-программист'], 'keyskills'..."
4,"Мужчина , 29 лет , родился 5 октября 1989",25000 руб.,Технический специалист,"Старый Оскол , не готов к переезду , не готов ...",полная занятость,"полный день, сменный график",Опыт работы 9 лет 9 месяцев Технический специ...,Комбинат хлебопродуктов Старооскольский,Слесарь КИПиА,Неоконченное высшее образование 2013 Воронежс...,22.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 9 лет 9 месяцев Август 2009 — по н...,moreThan6,['Комбинат хлебопродуктов Старооскольский www....,"[{'name': ['Слесарь'], 'keyskills': ['Рознична..."


In [96]:
path = '../data/mapping/names_to_region_id.json'

with open(path, 'r', encoding='utf-8') as file:
    cities_dict = json.load(file)

In [97]:
cities_list = list(cities_dict.keys())
cities_list[:10]

['Россия',
 'Виловатово',
 'Волжск',
 'Звенигово',
 'Знаменский',
 'Йошкар-Ола',
 'Кельмаксола',
 'Килемары',
 'Козьмодемьянск',
 'Красногорский (Республика Марий Эл)']

In [99]:
pattern = r'\b(?:' + '|'.join(re.escape(city) for city in cities) + r')\b'

def extract_city(text):
    match = re.search(pattern, text)
    
    return match.group(0) if match else None

In [87]:
data.iloc[0]['Город, переезд, командировки']

'Советск (Калининградская область) , не готов к переезду , не готов к командировкам'

In [105]:
data['city_clean'] = data['Город, переезд, командировки'].apply(extract_city)

In [106]:
def get_city_id(city):
    return cities_dict.get(city)

In [108]:
data['city_id'] = data['city_clean'].apply(get_city_id)
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,experience_clean,workExperience,experience,experience_dict,city_clean,city_id
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,Опыт работы 19 лет 5 месяцев Январь 2000 — по ...,moreThan6,['Временный трудовой коллектив Москва Информац...,"[{'name': ['Менеджер', 'Аналитик', 'Технически...",Королев,2090
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Опыт работы 10 лет 3 месяца Октябрь 2004 — Дек...,moreThan6,['ПАО Сбербанк Кассир-операционист Работа с де...,"[{'name': ['Кассир-операционист', 'Работа'], '...",Тверь,89
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,Опыт работы 18 лет 9 месяцев Август 2017 — Апр...,moreThan6,"['OpenSoft Саратов , www.opensoftdev.ru Информ...","[{'name': ['Инженер-программист'], 'keyskills'...",Саратов,79
4,"Мужчина , 29 лет , родился 5 октября 1989",25000 руб.,Технический специалист,"Старый Оскол , не готов к переезду , не готов ...",полная занятость,"полный день, сменный график",Опыт работы 9 лет 9 месяцев Технический специ...,Комбинат хлебопродуктов Старооскольский,Слесарь КИПиА,Неоконченное высшее образование 2013 Воронежс...,22.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 9 лет 9 месяцев Август 2009 — по н...,moreThan6,['Комбинат хлебопродуктов Старооскольский www....,"[{'name': ['Слесарь'], 'keyskills': ['Рознична...",Старый Оскол,1825
5,"Мужчина , 46 лет , родился 19 сентября 1972",250000 руб.,Руководитель ИТ-проектов,"Москва , не готов к переезду , готов к редким ...",полная занятость,полный день,Опыт работы 22 года 9 месяцев Руководитель ИТ...,"СИБИНТЕК, ООО ИК",Менеджер (Руководитель ИТ-проектов),Высшее образование 2008 ФГОУ ВПО «Уральская ак...,25.04.2019 22:48,Не указано,Опыт работы 22 года 9 месяцев Июнь 2015 — по н...,moreThan6,"['СИБИНТЕК, ООО ИК Москва , sibintek.ru Информ...","[{'name': ['Менеджер', 'Руководитель ИТ'], 'ke...",Москва,1


In [109]:
data['Занятость'].unique()

array(['частичная занятость, проектная работа, полная занятость',
       'полная занятость', 'частичная занятость, полная занятость',
       'частичная занятость',
       'проектная работа, частичная занятость, полная занятость',
       'частичная занятость, проектная работа',
       'проектная работа, полная занятость',
       'стажировка, полная занятость',
       'стажировка, частичная занятость, проектная работа',
       'проектная работа, частичная занятость',
       'стажировка, частичная занятость, проектная работа, полная занятость',
       'стажировка, частичная занятость, полная занятость',
       'стажировка, волонтерство, частичная занятость, проектная работа, полная занятость',
       'волонтерство, полная занятость', 'проектная работа',
       'проектная работа, волонтерство, полная занятость',
       'стажировка, проектная работа, полная занятость',
       'проектная работа, стажировка, частичная занятость, полная занятость'],
      dtype=object)

In [110]:
vacancies = pd.read_csv('vacancies_full_keyskills.csv')
vacancies['employment'].unique()

array(['full', 'part', 'probation', 'project', 'volunteer'], dtype=object)

In [131]:
employment_mapping = {
    'полная занятость': 'full',
    'неполный день': 'part',
    'частичная занятость': 'part',
    'стажировка': 'probation',
    'проектная работа': 'project',
    'волонтерство': 'volunteer'
}

In [135]:
def get_employment_mapping(employment):

    employment_list = employment.split(', ')
    employment_list = [employment_mapping.get(employment, None) for employment in employment_list]
    employment_list = [employment for employment in employment_list if employment is not None]

    return employment_list

data['employment'] = data['Занятость'].apply(get_employment_mapping)

In [137]:
type(data.iloc[0]['employment'])

list

In [138]:
data.iloc[0]['Занятость']

'частичная занятость, проектная работа, полная занятость'

In [140]:
data.iloc[0]['employment']

['part', 'project', 'full']

In [141]:
vacancies['workSchedule'].unique()

array(['fullDay', 'flyInFlyOut', 'shift', 'remote', 'flexible'],
      dtype=object)

In [142]:
data['График'].unique()

array(['гибкий график, полный день, сменный график, удаленная работа',
       'полный день', 'гибкий график, удаленная работа',
       'полный день, сменный график', 'удаленная работа',
       'полный день, удаленная работа', 'гибкий график, полный день',
       'гибкий график, полный день, удаленная работа',
       'гибкий график, полный день, сменный график, вахтовый метод, удаленная работа',
       'гибкий график, полный день, сменный график',
       'удаленная работа, гибкий график, полный день',
       'полный день, сменный график, удаленная работа',
       'удаленная работа, гибкий график, полный день, сменный график',
       'гибкий график, сменный график, удаленная работа',
       'удаленная работа, гибкий график', 'удаленная работа, полный день',
       'полный день, сменный график, вахтовый метод', 'гибкий график',
       'гибкий график, полный день, вахтовый метод',
       'полный день, вахтовый метод',
       'вахтовый метод, удаленная работа, гибкий график, полный день, см

In [143]:
workSchedule_mapping = {
    'вахтовый метод': 'flyInFlyOut',
    'удаленная работа': 'remote',
    'гибкий график': 'flexible',
    'сменный график': 'shift',
    'полный день': 'fullDay'
}

In [144]:
def get_workSchedule_mapping(workSchedule):

    workSchedule_list = workSchedule.split(', ')
    workSchedule_list = [workSchedule_mapping.get(workSchedule, None) for workSchedule in workSchedule_list]
    workSchedule_list = [workSchedule for workSchedule in workSchedule_list if workSchedule is not None]

    return workSchedule_list

data['workSchedule'] = data['График'].apply(get_workSchedule_mapping)

In [146]:
data.dropna(subset=['city_clean'], inplace=True)

In [147]:
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,experience_clean,workExperience,experience,experience_dict,city_clean,city_id,employment,workSchedule
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,Опыт работы 19 лет 5 месяцев Январь 2000 — по ...,moreThan6,['Временный трудовой коллектив Москва Информац...,"[{'name': ['Менеджер', 'Аналитик', 'Технически...",Королев,2090,"[part, project, full]","[flexible, fullDay, shift, remote]"
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Опыт работы 10 лет 3 месяца Октябрь 2004 — Дек...,moreThan6,['ПАО Сбербанк Кассир-операционист Работа с де...,"[{'name': ['Кассир-операционист', 'Работа'], '...",Тверь,89,[full],[fullDay]
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,Опыт работы 18 лет 9 месяцев Август 2017 — Апр...,moreThan6,"['OpenSoft Саратов , www.opensoftdev.ru Информ...","[{'name': ['Инженер-программист'], 'keyskills'...",Саратов,79,"[part, project, full]","[flexible, remote]"
4,"Мужчина , 29 лет , родился 5 октября 1989",25000 руб.,Технический специалист,"Старый Оскол , не готов к переезду , не готов ...",полная занятость,"полный день, сменный график",Опыт работы 9 лет 9 месяцев Технический специ...,Комбинат хлебопродуктов Старооскольский,Слесарь КИПиА,Неоконченное высшее образование 2013 Воронежс...,22.04.2019 15:59,Имеется собственный автомобиль,Опыт работы 9 лет 9 месяцев Август 2009 — по н...,moreThan6,['Комбинат хлебопродуктов Старооскольский www....,"[{'name': ['Слесарь'], 'keyskills': ['Рознична...",Старый Оскол,1825,[full],"[fullDay, shift]"
5,"Мужчина , 46 лет , родился 19 сентября 1972",250000 руб.,Руководитель ИТ-проектов,"Москва , не готов к переезду , готов к редким ...",полная занятость,полный день,Опыт работы 22 года 9 месяцев Руководитель ИТ...,"СИБИНТЕК, ООО ИК",Менеджер (Руководитель ИТ-проектов),Высшее образование 2008 ФГОУ ВПО «Уральская ак...,25.04.2019 22:48,Не указано,Опыт работы 22 года 9 месяцев Июнь 2015 — по н...,moreThan6,"['СИБИНТЕК, ООО ИК Москва , sibintek.ru Информ...","[{'name': ['Менеджер', 'Руководитель ИТ'], 'ke...",Москва,1,[full],[fullDay]


In [150]:
res_df = data[['ЗП',
               'Ищет работу на должность:',
               'city_clean',
               'city_id',
               'employment',
               'workSchedule',
               'workExperience', 
               'Последняя/нынешняя должность', 
               'experience_dict']]
res_df.head()

,ЗП,Ищет работу на должность:,city_clean,city_id,employment,workSchedule,workExperience,Последняя/нынешняя должность,experience_dict
1,40000 руб.,Технический писатель,Королев,2090,"[part, project, full]","[flexible, fullDay, shift, remote]",moreThan6,"Менеджер проекта, Аналитик, Технический писатель","[{'name': ['Менеджер', 'Аналитик', 'Технически..."
2,20000 руб.,Оператор,Тверь,89,[full],[fullDay],moreThan6,Кассир-операционист,"[{'name': ['Кассир-операционист', 'Работа'], '..."
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Саратов,79,"[part, project, full]","[flexible, remote]",moreThan6,Инженер-программист,"[{'name': ['Инженер-программист'], 'keyskills'..."
4,25000 руб.,Технический специалист,Старый Оскол,1825,[full],"[fullDay, shift]",moreThan6,Слесарь КИПиА,"[{'name': ['Слесарь'], 'keyskills': ['Рознична..."
5,250000 руб.,Руководитель ИТ-проектов,Москва,1,[full],[fullDay],moreThan6,Менеджер (Руководитель ИТ-проектов),"[{'name': ['Менеджер', 'Руководитель ИТ'], 'ke..."


In [152]:
res_df = res_df.rename(columns={
    'ЗП': 'compensation_from',
    'Ищет работу на должность:': 'name',
    'Последняя/нынешняя должность': 'last_name'
})

res_df = res_df[['name', 'compensation_from', 'city_clean', 'city_id', 
                 'employment', 'workSchedule', 'workExperience',
                 'last_name', 'experience_dict']]

res_df.head()

,name,compensation_from,city_clean,city_id,employment,workSchedule,workExperience,last_name,experience_dict
1,Технический писатель,40000 руб.,Королев,2090,"[part, project, full]","[flexible, fullDay, shift, remote]",moreThan6,"Менеджер проекта, Аналитик, Технический писатель","[{'name': ['Менеджер', 'Аналитик', 'Технически..."
2,Оператор,20000 руб.,Тверь,89,[full],[fullDay],moreThan6,Кассир-операционист,"[{'name': ['Кассир-операционист', 'Работа'], '..."
3,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,100000 руб.,Саратов,79,"[part, project, full]","[flexible, remote]",moreThan6,Инженер-программист,"[{'name': ['Инженер-программист'], 'keyskills'..."
4,Технический специалист,25000 руб.,Старый Оскол,1825,[full],"[fullDay, shift]",moreThan6,Слесарь КИПиА,"[{'name': ['Слесарь'], 'keyskills': ['Рознична..."
5,Руководитель ИТ-проектов,250000 руб.,Москва,1,[full],[fullDay],moreThan6,Менеджер (Руководитель ИТ-проектов),"[{'name': ['Менеджер', 'Руководитель ИТ'], 'ke..."


In [153]:
res_df.to_csv('validation_dataset.csv', index=False)